[View in Colaboratory](https://colab.research.google.com/github/idleuncle/Colab/blob/master/ColabTutorial.ipynb)

# Colab Tutorial

## 安装依赖

In [1]:

!pip install keras
!pip install jieba
!pip install h5py

    100% |████████████████████████████████| 7.3MB 4.4MB/s 
  Running setup.py bdist_wheel for jieba ... - \ | / - \ | / done
  Stored in directory: /content/.cache/pip/wheels/c9/c7/63/a9ec0322ccc7c365fd51e475942a82395807186e94f0522243
Successfully built jieba


In [0]:
import sys, os
import h5py
import jieba as jb
import numpy as np
import keras as krs
import tensorflow as tf
from sklearn.preprocessing import LabelEncoder


## 加载数据

### 授权登录

In [0]:
# 安装 PyDrive 操作库，该操作每个 notebook 只需要执行一次
!pip install -U -q PyDrive
from pydrive.auth import GoogleAuth
from pydrive.drive import GoogleDrive
from google.colab import auth
from oauth2client.client import GoogleCredentials

def login_google_drive():
  # 授权登录，仅第一次的时候会鉴权
  auth.authenticate_user()
  gauth = GoogleAuth()
  gauth.credentials = GoogleCredentials.get_application_default()
  drive = GoogleDrive(gauth)
  return drive

drive = login_google_drive()

### 列出Google Driver上文件

In [34]:
def gd_ls(drive):
  file_list = drive.ListFile({'q': "'root' in parents and trashed=false"}).GetList()
  for file1 in file_list:
    print('title: %s, id: %s, mimeType: %s' % (file1['title'], file1['id'], file1["mimeType"]))
    

gd_ls(drive)

title: design.txt, id: 1OS1BuBHxtJWSvcLiWnsdg1VzxjxokPH3, mimeType: text/plain
title: health.txt, id: 1JCAq9rdUD8OsKUtypp70b8gpFminTFmU, mimeType: text/plain
title: tech.txt, id: 1YSqi5II82Fkb7bMAeHrhB_ogVcSvdnTs, mimeType: text/plain
title: Colab Notebooks, id: 1WZgWzW1T9OWVoxQXHd4mVwJwZApwia6v, mimeType: application/vnd.google-apps.folder
title: _Moo.do Files (Do not delete), id: 0BzfMfhiuy0xGdkVER0N2TVRacnc, mimeType: application/vnd.google-apps.folder
title: SimpleMind, id: 0BzfMfhiuy0xGMHlqNDJEbGRIUzA, mimeType: application/vnd.google-apps.folder
title: Untitled Diagram.html, id: 0BzfMfhiuy0xGakY0UmppZDlkY3M, mimeType: application/vnd.jgraph.mxfile.realtime
title: Untitled Diagram.html, id: 0BzfMfhiuy0xGelNkc1lXQ2ZVNTA, mimeType: application/vnd.jgraph.mxfile.realtime
title: Untitled Diagram.html, id: 0BzfMfhiuy0xGLVJBRXV4UTdNZ0E, mimeType: application/vnd.jgraph.mxfile.realtime


### 拷贝Google Drive上文件至Colab

In [0]:
import os
def download_from_google_drive(drive, file_name_prefix, colab_dir=".", overwrite=True):
  # choose a local (colab) directory to store the data.
  local_download_path = os.path.expanduser(colab_dir)
  try:
    os.makedirs(local_download_path)
  except: pass

  # 2. Auto-iterate using the query syntax
  #    https://developers.google.com/drive/v2/web/search-parameters
  file_list = drive.ListFile(
      {'q': "title contains '%s'" % (file_name_prefix) }).GetList()

  files_dict = {}
  for f in file_list:
    # 3. Create & download by id.
    print('title: %s, id: %s' % (f['title'], f['id']))
    fname = os.path.join(local_download_path, f['title'])
    if overwrite or not os.path.exists(fname):
      print('downloading to {}'.format(fname))
      f_ = drive.CreateFile({'id': f['id']})
      f_.GetContentFile(fname)
      print('Download Completed!')
    files_dict[ f['title'] ] = fname

  # with open(fname, 'r') as f:
  #   print(f.read())
  return files_dict, local_download_path

In [15]:
print(download_from_google_drive(drive, '.py'))

title: IpynbModules.py, id: 1cpFFQi3aBG_yEuWcKLRxYfvxFxHzY5h3
downloading to ./IpynbModules.py
Download Completed!
title: IpynbImporter.py, id: 1qKLyot96gRiUjxHJMsJXHCgqYAXLx7ZA
downloading to ./IpynbImporter.py
Download Completed!
({u'IpynbModules.py': u'./IpynbModules.py', u'IpynbImporter.py': u'./IpynbImporter.py'}, '.')


In [0]:
import IpynbImporter
sys.meta_path

In [29]:
print(download_from_google_drive(drive, 'ColabModules.ipynb'))

title: ColabModules.ipynb, id: 1IMv93f2bMYhrx2lfL3cmDBI7kmjCMy01
downloading to ./ColabModules.ipynb
Download Completed!
({u'ColabModules.ipynb': u'./ColabModules.ipynb'}, '.')


In [30]:
!ls

ColabModules.ipynb  IpynbImporter.py   IpynbModules.py
datalab		    IpynbImporter.pyc  IpynbModules.pyc


In [31]:
import ColabModules
ColabModules.foo()

importing Jupyter notebook from ColabModules.ipynb


In [18]:
with open('/content/IpynbModules.py', 'r') as f:
  print(f.read())

def foo():
    print("foo")



### 缓存数据到工作环境

In [7]:
def cache_data():
  # id 替换成上一步读取到的对应文件 id
  health_txt = drive.CreateFile({'id': "1JCAq9rdUD8OsKUtypp70b8gpFminTFmU"}) 
  tech_txt = drive.CreateFile({'id': "1YSqi5II82Fkb7bMAeHrhB_ogVcSvdnTs"})
  design_txt = drive.CreateFile({'id': "1OS1BuBHxtJWSvcLiWnsdg1VzxjxokPH3"})
  #这里的下载操作只是缓存，不会在你的Google Drive 目录下多下载一个文件
  
  health_txt.GetContentFile('health.txt', "text/plain")
  tech_txt.GetContentFile('tech.txt', "text/plain")
  design_txt.GetContentFile('design.txt', "text/plain")
  
  print("缓存成功")
  
cache_data()



缓存成功


### 读取工作环境数据

In [8]:
def load_data():
    titles = []
    print("正在加载健康类别的数据...")
    with open("health.txt", "r") as f:
        for line in f.readlines():
            titles.append(line.strip())

    print("正在加载科技类别的数据...")
    with open("tech.txt", "r") as f:
        for line in f.readlines():
            titles.append(line.strip())


    print("正在加载设计类别的数据...")
    with open("design.txt", "r") as f:
        for line in f.readlines():
            titles.append(line.strip())

    print("一共加载了 %s 个标题" % len(titles))

    return titles
  
titles = load_data()


正在加载健康类别的数据...
正在加载科技类别的数据...
正在加载设计类别的数据...
一共加载了 31318 个标题


### 加载标签

In [9]:
def load_label():
    arr0 = np.zeros(shape=[12000, ])
    arr1 = np.ones(shape=[12000, ])
    arr2 = np.array([2]).repeat(7318)
    target = np.hstack([arr0, arr1, arr2])
    print("一共加载了 %s 个标签" % target.shape)

    encoder = LabelEncoder()
    encoder.fit(target)
    encoded_target = encoder.transform(target)
    dummy_target = krs.utils.np_utils.to_categorical(encoded_target)

    return dummy_target
  
target = load_label()


一共加载了 31318 个标签


## 文本预处理

In [0]:
max_sequence_length = 30
embedding_size = 50

# 标题分词
titles = [".".join(jb.cut(t, cut_all=True)) for t in titles]

# �word2vec 词袋化
vocab_processor = tf.contrib.learn.preprocessing.VocabularyProcessor(max_sequence_length, min_frequency=1)
text_processed = np.array(list(vocab_processor.fit_transform(titles)))

# 读取词标签
dict = vocab_processor.vocabulary_._mapping
sorted_vocab = sorted(dict.items(), key = lambda x : x[1])


## 构建神经网络

In [18]:
# 配置网络结构
def build_netword(num_vocabs):
    # 配置网络结构
    model = krs.Sequential()
    model.add(krs.layers.Embedding(num_vocabs, embedding_size, input_length=max_sequence_length))
    model.add(krs.layers.LSTM(32, dropout=0.2, recurrent_dropout=0.2))
    model.add(krs.layers.Dense(3))
    model.add(krs.layers.Activation("softmax"))
    model.compile(loss="categorical_crossentropy", optimizer="adam", metrics=["accuracy"])

    return model
  
num_vocabs = len(dict.items())
model = build_netword(num_vocabs=num_vocabs)

import time
start = time.time()
# 训练模型
model.fit(text_processed, target, batch_size=512, epochs=10, )
finish = time.time()
print("Traning time %d s." % (finish - start))


Epoch 1/10
31318/31318 [==============================] - 6s 199us/step - loss: 0.9805 - acc: 0.4480
Epoch 2/10
31318/31318 [==============================] - 5s 159us/step - loss: 0.6357 - acc: 0.6197
Epoch 3/10
31318/31318 [==============================] - 5s 159us/step - loss: 0.5140 - acc: 0.7585
Epoch 4/10
31318/31318 [==============================] - 5s 160us/step - loss: 0.2544 - acc: 0.9247
Epoch 5/10
31318/31318 [==============================] - 5s 160us/step - loss: 0.1648 - acc: 0.9548
Epoch 6/10
 5632/31318 [====>.........................] - ETA: 4s - loss: 0.1300 - acc: 0.9648

31318/31318 [==============================] - 5s 160us/step - loss: 0.1175 - acc: 0.9691
Epoch 7/10
31318/31318 [==============================] - 5s 159us/step - loss: 0.0875 - acc: 0.9779
Epoch 8/10
31318/31318 [==============================] - 5s 160us/step - loss: 0.0735 - acc: 0.9819
Epoch 9/10
31318/31318 [==============================] - 5s 161us/step - loss: 0.0573 - acc: 0.9866
Epoch 10/10
31318/31318 [==============================] - 5s 159us/step - loss: 0.0534 - acc: 0.9879
Traning time 53 s.


## 预测样本

In [19]:
#@title Default title text
sen = "# \\u5B89\\u88C5 PyDrive \\u64CD\\u4F5C\\u5E93\\uFF0C\\u8BE5\\u64CD\\u4F5C\\u6BCF\\u4E2A notebook \\u53EA\\u9700\\u8981\\u6267\\u884C\\u4E00\\u6B21" #@param ["# \u5B89\u88C5 PyDrive \u64CD\u4F5C\u5E93\uFF0C\u8BE5\u64CD\u4F5C\u6BCF\u4E2A notebook \u53EA\u9700\u8981\u6267\u884C\u4E00\u6B21", "\u907F\u514D\u66B4\u996E\u66B4\u98DF\uFF0C\u665A\u4E0A\u4E5D\u70B9\u4E4B\u540E\u4E5F\u6700\u597D\u4E0D\u8981\u5403\u96F6\u98DF\u3002", "\u5403\u65B9\u4FBF\u9762\u7684\u65F6\u5019\uFF0C\u5148\u7528\u6E29\u6C34\u6765\u6D17\u51C0\u65B9\u4FBF\u9762\uFF0C\u6405\u62CC\u540E\u5012\u6389\u6709\u8721\u6D82\u5C42\u7684\u6C34\uFF0C\u91CD\u590D\u540E\u5728\u7528\u53E6\u4E00\u58F6\u6C34\u716E\u9762\uFF0C\u56E0\u4E3A\u65B9\u4FBF\u9762\u6709\u4E00\u5C42\u8721\u6D82\u5C42\uFF0C\u5927\u91CF\u79EF\u7D2F\u5728\u4F53\u5185\u4F1A\u81F4\u764C", "\u706B\u661F\u79FB\u6C11\u8BA1\u5212\uFF08Mars migration program ;Mars immigrant plans \uFF09\u662F\u7F8E\u56FD\u5B87\u5B99\u63A2\u7D22\u6280\u672F\u516C\u53F8(SpaceX)\u603B\u88C1\u517C\u521B\u59CB\u4EBA\u57C3\u9686\xB7\u9A6C\u65AF\u514B\u9996\u6B21\u5BF9\u5A92\u4F53\u900F\u9732\u51FA\u7684\u3001\u79FB\u6C11\u706B\u661F\u5E76\u5728\u706B\u661F\u5EFA\u793E\u533A\u7684\u8BA1\u5212\u3002\u5177\u4F53\u662F\u5426\u53EF\u884C\u4EE5\u53CA\u706B\u661F\u6539\u9020\u8BA1\u5212\u3001\u5173\u4E8E\u706B\u661F\u6863\u6848\u76F8\u5173\u5929\u6587\u5B66\u8005\u3001\u79D1\u5B66\u5BB6\u3001\u5A92\u4F53\u7ED9\u51FA\u4E0D\u540C\u770B\u6CD5\u3002"] {allow-input: true}
#sen = "做好商业设计需要学习的小技巧"
#sen = "避免暴饮暴食，晚上九点之后也最好不要吃零食。"
#sen = "吃方便面的时候，先用温水来洗净方便面，搅拌后倒掉有蜡涂层的水，重复后在用另一壶水煮面，因为方便面有一层蜡涂层，大量积累在体内会致癌"
#sen = "火星移民计划（Mars migration program ;Mars immigrant plans ）是美国宇宙探索技术公司(SpaceX)总裁兼创始人埃隆·马斯克首次对媒体透露出的、移民火星并在火星建社区的计划。具体是否可行以及火星改造计划、关于火星档案相关天文学者、科学家、媒体给出不同看法。"
sen_prosessed = " ".join(jb.cut(sen, cut_all=True))
sen_prosessed = vocab_processor.transform([sen_prosessed])
sen_prosessed = np.array(list(sen_prosessed))
result = model.predict(sen_prosessed)

catalogue = list(result[0]).index(max(result[0]))
threshold=0.8
if max(result[0]) > threshold:
    if catalogue == 0:
        print("这是一篇关于健康的文章")
    elif catalogue == 1:
        print("这是一篇关于科技的文章")
    elif catalogue == 2:
        print("这是一篇关于设计的文章")
    else:
        print("这篇文章没有可信分类")


这是一篇关于科技的文章
